# BMSB Simulation

This Notebook calculates the 'Transfer Probability' (huff Model) for each city pair and updates the 'MN_Cities_Pts_WGS' table for each simulation based on randomly asking if the transfer probability is greater than a random value (if yes assign presence (1) if no, assign absence (0)). This process will be repeated 60 times for each simulaton (to apporximate a monthly interval over 5 years, the temporal length of BMSB observation reocrded in our data). Finally, the simulations are run 100 times to best assess the general trend of BMSB spreading to cities across MN. 

In [28]:
import arcpy
import random
import time

# Functions

In [29]:
def getPresence(fc, fields, SimCol):
    '''
    looking through current simulation layer and organizes cities into Absence or Presence lists
    '''
    presenceList = []
    absenceList = []

    with arcpy.da.SearchCursor(fc, fields) as cursor:
        for row in cursor:
            if row[2] == 1:
                presenceList.append(row[0])
            else:
                absenceList.append(row[0])
    return presenceList, absenceList


In [30]:
def runModel(fc, fields, presenceList, absenceList):
    '''
    Uses presence and absence list, to check the relationship between each pair of cities (we only care to check 
    cities with 0-1 or 1-0). If we find Cities pairs that meet this requirement, then we test its Huff probability against a 
    random number to determine if the destCity will now be presence.
    
    returns a list of Cities that BMSB spreads to
    '''
    
    spreadList = [] #initialize spreadList
    
    
    with arcpy.da.SearchCursor(fc, fields) as cursor:

        for row in cursor:
            if row[0] in presenceList:
                destCity = row[1]

            elif row[1] in presenceList:
                destCity = row[0]

            else:
                continue

            TransProb = row[2]

            if destCity in absenceList:
#                 print('destCity TransProb is being tested')
                if TransProb >= random.random():
                    spreadList.append(destCity)         
    
    return spreadList

In [31]:
def updateSim(fc, fields, spreadList):
    '''
    using the returned 'Spreadlist' to update MN_CIties_Pts_WGS
    '''
    
    with arcpy.da.UpdateCursor(fc, fields) as cursor:
        for row in cursor:
            if row[0] in spreadList:
                row[1] = 1
                cursor.updateRow(row)

In [32]:
def calculateHuff(fc, fields, alpha):
    '''
    calculates the transfer probabilities (updates the "TransProb" column in 'MN_Cities_Pts_NearTable')
    '''
    HuffDenominator = 0
    
    #first calculate HuffDenominator from sum of HuffNumerators in city pairs
    # looking through Neartable
    with arcpy.da.SearchCursor(fc, fields) as cursor:
        
        for row in cursor:
            
            #assign values
            city1Pop = row[3]
            city2Pop = row[4]
            distance = row[0]
            
            #calculate HuffNumerator and add to HuffDenominator
            HuffNumerator = ((city1Pop * city2Pop) / (distance ** alpha))
            HuffDenominator += HuffNumerator
    
    #use UpdateCursor to calculate and assign probability of transfer for city pairs
    with arcpy.da.UpdateCursor(fc, fields) as cursor:
        
        for row in cursor:
            
            #assign values
            city1Pop = row[3]
            city2Pop = row[4]
            distance = row[0]
            
            #calculate transProb and update row
            HuffNumerator = ((city1Pop * city2Pop) / (distance ** alpha))
            
            #emulates the idea of running daily timesteps (without having to run 1800 timesteps for true daily outputs)
            scalar = 30 # SET SCALAR MANUALLY
            
            row[5] = scalar * (HuffNumerator / HuffDenominator)
            cursor.updateRow(row)

# Main

In [35]:
#Calculates and Stores 'Transfer Probabilities' for all city pairs

fc = 'MN_Cities_Pts_NearTable'
fields = ['NEAR_DIST', 'CITY_1', 'CITY_2','CITY_1_POP', 'CITY_2_POP', 'TransProb']
alpha = 2

print('Running...')
calculateHuff(fc, fields, alpha)
print('Complete')

Running...
Complete


In [48]:
start_time = time.time()

#run 100 simulations
for i in range(0,100):
    
    SimCol = 'BMSB_Sim'+str(i)


    #populate Sim column with seed city
    fc = 'MN_Cities_Pts_WGS'
    seed = "Saint Paul" #SEED CITY DEFINED MANUALLY
    fields = ['CITY_SJ', 'POPULATION', SimCol]

    with arcpy.da.UpdateCursor(fc, fields) as cursor:
        for row in cursor:
            if row[0] == seed:
                row[2] = 1
            else:
                row[2] = 0
            cursor.updateRow(row)

    timestep = 60 #SET NUMBER OF TIMESTEPS
                    # 5 years = 60months = 260 weeks = 1825 days

    for k in range(0, timestep):

        fc = 'MN_Cities_Pts_WGS'
        fields = ['CITY_SJ', 'POPULATION', SimCol]

        presenceList, absenceList = getPresence(fc, fields, SimCol)

        # simulate spread in a single timestep
        fc = 'MN_Cities_Pts_NearTable'
        fields = ['CITY_1', 'CITY_2','TransProb']

        spreadList = runModel(fc, fields, presenceList, absenceList)
        print("Sim " +str(i) +" | Step " + str(k) + ": " + str(spreadList))

        # update spread in a single timestep
        fc = 'MN_Cities_Pts_WGS'
        fields = ['CITY_SJ', SimCol]

        updateSim(fc, fields, spreadList)

    print('Simulation Complete')
print("All Simulations Complete")
print("--- %s seconds ---" % (time.time() - start_time))

Sim 0 | Step 0: ['Maplewood', 'Minneapolis']
Sim 0 | Step 1: ['Buffalo', 'Eagan', 'Eden Prairie', 'Edina', 'Arden Hills', 'Bloomington', 'Plymouth', 'Brooklyn Park', 'Blaine', 'Apple Valley', 'Osseo', 'Richfield', 'Saint Louis Park', 'West Saint Paul', 'White Bear Lake', 'Shakopee', 'Woodbury']
Sim 0 | Step 2: ['Lino Lakes', 'Fridley', 'Andover', 'Burnsville', 'Champlin', 'Hopkins', 'Inver Grove Heights', 'Little Canada', 'Maple Grove', 'North Saint Paul', 'Robbinsdale', 'Minnetonka', 'Champlin', 'Savage', 'Zimmerman']
Sim 0 | Step 3: ['Anoka', 'Coon Rapids', 'Coon Rapids', 'Crystal', 'Golden Valley', 'New Brighton', 'Lakeville', 'Lakeville', 'Roseville', 'Roseville', 'Lexington', 'Roseville', 'New Brighton', 'South Saint Paul', 'New Hope', 'Vadnais Heights']
Sim 0 | Step 4: ['Chaska', 'Corcoran', 'Ramsey', 'Hermantown', 'Rosemount', 'Wayzata', 'Mounds View', 'Mounds View', 'Oakdale', 'Oakdale', 'Prior Lake', 'Oakdale', 'Oakdale']
Sim 0 | Step 5: ['Ham Lake', 'Duluth', 'Lake Elmo', 'Ch

Sim 4 | Step 48: ['Saint Augusta']
Sim 4 | Step 49: ['Wyoming', 'Afton']
Sim 4 | Step 50: []
Sim 4 | Step 51: []
Sim 4 | Step 52: []
Sim 4 | Step 53: ['Vermillion']
Sim 4 | Step 54: ['Sauk Centre']
Sim 4 | Step 55: ['Owatonna']
Sim 4 | Step 56: ['Lindstrom', 'Mendota']
Sim 4 | Step 57: ['Bayport']
Sim 4 | Step 58: ['Saint Joseph']
Sim 4 | Step 59: []
Simulation Complete
Sim 5 | Step 0: ['Kasson', 'Maplewood', 'Oakdale', 'Minneapolis', 'Bloomington', 'Stillwater']
Sim 5 | Step 1: ['Woodbury', 'Plymouth', 'Cambridge', 'Osseo', 'Saint Anthony', 'Saint Louis Park', 'West Saint Paul', 'Roseville', 'Vadnais Heights', 'Woodbury']
Sim 5 | Step 2: ['Lakeville', 'Inver Grove Heights', 'Maple Grove', 'Columbia Heights', 'Hopkins', 'Coon Rapids', 'Chaska', 'New Brighton', 'South Saint Paul', 'East Bethel', 'Maple Grove']
Sim 5 | Step 3: ['Burnsville', 'Champlin', 'Eagan', 'Golden Valley', 'Cottage Grove', 'Savage', 'Mounds View', 'Minnetonka', 'Champlin', 'Saint Paul Park']
Sim 5 | Step 4: ['Apple

Sim 9 | Step 16: ['Monticello', 'Mahtomedi', 'Grant', 'Rogers', 'New Prague']
Sim 9 | Step 17: ['Delano', 'Corcoran']
Sim 9 | Step 18: ['Greenfield', 'Faribault', 'Mound']
Sim 9 | Step 19: ['Forest Lake', 'Montgomery', 'Norwood Young America', 'Scandia']
Sim 9 | Step 20: ['Wyoming', 'Gilbert']
Sim 9 | Step 21: ['Pine River', 'Circle Pines', 'Willernie']
Sim 9 | Step 22: ['Coates', 'Landfall']
Sim 9 | Step 23: ['Watertown']
Sim 9 | Step 24: ['Shorewood']
Sim 9 | Step 25: []
Sim 9 | Step 26: ['Tonka Bay']
Sim 9 | Step 27: []
Sim 9 | Step 28: ['Osseo', 'Duluth']
Sim 9 | Step 29: ['Saint Cloud']
Sim 9 | Step 30: ['Saint Bonifacius', 'Pipestone']
Sim 9 | Step 31: ['Sunfish Lake']
Sim 9 | Step 32: []
Sim 9 | Step 33: []
Sim 9 | Step 34: ['Sartell']
Sim 9 | Step 35: []
Sim 9 | Step 36: []
Sim 9 | Step 37: []
Sim 9 | Step 38: ['Henderson', 'Owatonna']
Sim 9 | Step 39: ['Foley', 'Waconia']
Sim 9 | Step 40: ['Belle Plaine', 'Belle Plaine']
Sim 9 | Step 41: []
Sim 9 | Step 42: []
Sim 9 | Step 43:

Sim 14 | Step 4: ['Apple Valley', 'Eagan', 'Credit River', 'Forest Lake', 'New Brighton', 'Golden Valley', 'Grant', 'Golden Valley', 'Lino Lakes', 'Crystal', 'New Brighton', 'Willernie', 'Rogers']
Sim 14 | Step 5: ['Anoka', 'Lakeville', 'Shakopee', 'Circle Pines', 'Lake Elmo', 'Independence', 'Hanover', 'New Hope', 'Chaska']
Sim 14 | Step 6: ['Cambridge', 'Greenfield', 'Saint Michael', 'Lauderdale', 'Arden Hills', 'Nowthen', 'Savage']
Sim 14 | Step 7: ['Centerville', 'Champlin', 'Falcon Heights', 'Lilydale', 'Falcon Heights', 'Newport', 'Ham Lake']
Sim 14 | Step 8: ['Albertville', 'Ramsey', 'Chanhassen', 'Dayton', 'Hastings', 'Afton']
Sim 14 | Step 9: ['Rosemount', 'Inver Grove Heights', 'Inver Grove Heights', 'Farmington', 'Saint Francis', 'Birchwood Village', 'Woodland']
Sim 14 | Step 10: ['Elk River', 'Mendota Heights', 'Prior Lake', 'Orono', 'Saint Paul Park', 'Prior Lake']
Sim 14 | Step 11: ['East Bethel', 'Isanti', 'Corcoran', 'Mound', 'Saint Cloud', 'Watertown']
Sim 14 | Step 12

Sim 18 | Step 47: ['Waite Park']
Sim 18 | Step 48: ['Dundas', 'Monticello', 'Hanover']
Sim 18 | Step 49: ['Courtland', 'Milaca']
Sim 18 | Step 50: ['Waconia']
Sim 18 | Step 51: ['Albertville', 'Dellwood']
Sim 18 | Step 52: []
Sim 18 | Step 53: ['Zimmerman']
Sim 18 | Step 54: ['Annandale', 'Rockville']
Sim 18 | Step 55: ['Bertha']
Sim 18 | Step 56: ['Isanti']
Sim 18 | Step 57: []
Sim 18 | Step 58: []
Sim 18 | Step 59: []
Simulation Complete
Sim 19 | Step 0: ['Cologne', 'Minneapolis', 'White Bear Lake', 'Bloomington', 'Ham Lake']
Sim 19 | Step 1: ['Andover', 'Blaine', 'Eden Prairie', 'Golden Valley', 'Little Canada', 'Long Lake', 'Mahtomedi', 'Edina', 'Saint Louis Park', 'West Saint Paul', 'Shakopee', 'Shoreview']
Sim 19 | Step 2: ['Richfield', 'Burnsville', 'Columbia Heights', 'Eagan', 'Hugo', 'Maplewood', 'Richfield', 'Brooklyn Park', 'Apple Valley', 'Mound', 'Eagan', 'Mounds View', 'Apple Valley', 'Woodbury']
Sim 19 | Step 3: ['Anoka', 'Otsego', 'Coon Rapids', 'Plymouth', 'Hastings', 

...

Sim 79 | Step 58: ['Litchfield', 'Atwater']
Sim 79 | Step 59: []
Simulation Complete
Sim 80 | Step 0: ['Falcon Heights', 'West Saint Paul', 'North Saint Paul', 'Minneapolis']
Sim 80 | Step 1: ['Farmington', 'Maplewood', 'Richfield', 'Bloomington', 'Maplewood', 'Inver Grove Heights', 'Richfield', 'Saint Louis Park', 'Oakdale', 'Bloomington', 'Shoreview']
Sim 80 | Step 2: ['Blaine', 'Columbia Heights', 'Edina', 'Edina', 'Brooklyn Center', 'Plymouth', 'Wayzata', 'Savage', 'Minnetonka', 'Plymouth', 'Eden Prairie', 'South Saint Paul', 'Columbia Heights', 'Cottage Grove', 'Brooklyn Center', 'Vadnais Heights', 'Woodbury']
Sim 80 | Step 3: ['Brooklyn Park', 'Buffalo', 'Burnsville', 'Chanhassen', 'New Hope', 'Deephaven', 'Eagan', 'Eagan', 'Eagan', 'Robbinsdale', 'Excelsior', 'Golden Valley', 'Lauderdale', 'Roseville', 'New Brighton', 'Roseville', 'Hopkins', 'Shakopee']
Sim 80 | Step 4: ['Apple Valley', 'Osseo', 'Byron', 'Coon Rapids', 'Lake Elmo', 'Lakeville', 'Spring Lake Park', 'Crystal', 'Sc

Sim 84 | Step 35: []
Sim 84 | Step 36: ['Dayton']
Sim 84 | Step 37: ['Afton']
Sim 84 | Step 38: ['Greenfield']
Sim 84 | Step 39: []
Sim 84 | Step 40: ['Albertville', 'South Haven', 'Maple Plain', 'Oak Park Heights']
Sim 84 | Step 41: []
Sim 84 | Step 42: ['Oak Grove', 'Richmond', 'Rochester']
Sim 84 | Step 43: ['Wyoming']
Sim 84 | Step 44: ['Austin', 'Lester Prairie', 'Wanamingo']
Sim 84 | Step 45: ['Saint Michael', 'Sartell']
Sim 84 | Step 46: ['Litchfield', 'Darwin']
Sim 84 | Step 47: ['Coates']
Sim 84 | Step 48: ['Hanover']
Sim 84 | Step 49: ['Le Center', 'Saint Cloud']
Sim 84 | Step 50: ['Dundas']
Sim 84 | Step 51: ['Sunfish Lake']
Sim 84 | Step 52: []
Sim 84 | Step 53: ['Pine Springs']
Sim 84 | Step 54: ['Lexington']
Sim 84 | Step 55: ['Big Lake']
Sim 84 | Step 56: ['Carver']
Sim 84 | Step 57: ['Rush City', 'Landfall', 'Rush City']
Sim 84 | Step 58: ['Cold Spring', 'Saint Peter', 'Sleepy Eye']
Sim 84 | Step 59: []
Simulation Complete
Sim 85 | Step 0: ['Edina', 'Inver Grove Heights

Sim 89 | Step 8: ['Mahtomedi', 'North Mankato', 'Ramsey', 'Cottage Grove', 'Mound']
Sim 89 | Step 9: ['Stillwater', 'Wyoming']
Sim 89 | Step 10: ['Anoka', 'Cambridge', 'Lakeville', 'Lino Lakes', 'Mounds View', 'Wayzata', 'Ham Lake']
Sim 89 | Step 11: ['Deephaven', 'Deephaven', 'Brainerd', 'Lauderdale', 'Falcon Heights', 'Spring Lake Park', 'Tonka Bay']
Sim 89 | Step 12: ['Greenfield', 'Medina', 'Northfield', 'Oak Park Heights']
Sim 89 | Step 13: ['Albertville', 'Farmington', 'Hutchinson', 'Spring Park']
Sim 89 | Step 14: ['Rogers']
Sim 89 | Step 15: ['Prior Lake', 'Elko New Market']
Sim 89 | Step 16: ['Chisago City', 'Saint Paul Park', 'Otsego']
Sim 89 | Step 17: ['Dayton']
Sim 89 | Step 18: ['Lonsdale', 'Osseo', 'Grant']
Sim 89 | Step 19: ['East Bethel']
Sim 89 | Step 20: ['Columbus', 'Rochester']
Sim 89 | Step 21: ['Faribault']
Sim 89 | Step 22: []
Sim 89 | Step 23: ['Waconia']
Sim 89 | Step 24: ['Lexington', 'Hastings']
Sim 89 | Step 25: ['Centerville', 'Circle Pines']
Sim 89 | Step

Sim 94 | Step 1: ['Hopkins', 'Brooklyn Center', 'Fridley', 'Chanhassen', 'Lakeville', 'Mounds View', 'New Brighton', 'New Brighton']
Sim 94 | Step 2: ['Eden Prairie', 'Columbia Heights', 'Eagan', 'Eden Prairie', 'Golden Valley', 'Oakdale', 'Robbinsdale', 'Roseville', 'New Hope', 'Plymouth', 'Brooklyn Park', 'Maple Grove', 'Center City', 'Little Canada', 'Brooklyn Park', 'Saint Louis Park', 'Saint Louis Park', 'West Saint Paul', 'Mendota Heights', 'Roseville', 'White Bear Lake', 'Shoreview', 'Woodbury']
Sim 94 | Step 3: ['Blaine', 'Lino Lakes', 'Coon Rapids', 'Byron', 'Minnetonka', 'Chaska', 'Cottage Grove', 'Crystal', 'Inver Grove Heights', 'Edina', 'Bloomington', 'Oak Grove', 'Richfield', 'Saint Anthony', 'Crystal']
Sim 94 | Step 4: ['Belle Plaine', 'Burnsville', 'Elko New Market', 'Elk River', 'Mahtomedi', 'Falcon Heights', 'Dayton', 'Red Wing', 'Lilydale', 'Lauderdale', 'Savage', 'South Saint Paul', 'Spring Lake Park']
Sim 94 | Step 5: ['Hilltop', 'Ramsey', 'Andover', 'Ramsey', 'Ard

Sim 98 | Step 32: ['Carver', 'Granite Falls', 'Red Wing', 'Zumbro Falls']
Sim 98 | Step 33: ['Clara City']
Sim 98 | Step 34: ['Oak Grove']
Sim 98 | Step 35: ['Excelsior', 'Norwood Young America']
Sim 98 | Step 36: ['Elk River', 'Corcoran', 'Saint James', 'Saint Bonifacius', 'Sunfish Lake']
Sim 98 | Step 37: ['Mayer']
Sim 98 | Step 38: []
Sim 98 | Step 39: ['Big Lake', 'Tonka Bay']
Sim 98 | Step 40: []
Sim 98 | Step 41: ['Rice Lake', 'New Ulm', 'Orono', 'Sauk Rapids']
Sim 98 | Step 42: []
Sim 98 | Step 43: []
Sim 98 | Step 44: []
Sim 98 | Step 45: ['Waseca']
Sim 98 | Step 46: ['Minnetrista']
Sim 98 | Step 47: ['Stacy']
Sim 98 | Step 48: []
Sim 98 | Step 49: []
Sim 98 | Step 50: ['Center City']
Sim 98 | Step 51: []
Sim 98 | Step 52: ['Hanover']
Sim 98 | Step 53: ['Elko New Market']
Sim 98 | Step 54: ['Hutchinson']
Sim 98 | Step 55: ['North Mankato']
Sim 98 | Step 56: ['Albertville', 'Wyoming']
Sim 98 | Step 57: []
Sim 98 | Step 58: ['Mankato', 'Credit River']
Sim 98 | Step 59: ['Hibbing'